## feature selection algorithm:

In [1]:
import pandas as pd

data = pd.read_csv('data_sample_three_years.csv')

data = data.iloc[:, [0,1,2,3,4]]
data.rename(columns={'date': 'open', 'low':'close', 'open':'high', 'high':'low'}, inplace=True)

data['date'] = pd.to_datetime(data['timestamp'], unit='s')
data = data.iloc[:, [0,5,1,2,3,4]]

data_backup = data.loc[ data.timestamp >= 1675814400 ]
data = data.loc[ data.timestamp < 1675814400 ]

from talib import RSI
data['rsi_14'] = RSI(data['close'], timeperiod=14)

from talib import MA, SMA, EMA
data['ma_9'] = MA(data['close'], timeperiod=9)

from talib import MACD
data['macd'], data['signal'], data['hist'] = MACD(data['close'])

from talib import ADX
data['adx'] = ADX(data['high'], data['low'], data['close'])

data.dropna(axis=0, inplace=True)

In [2]:
data['close_roc'] = ( data['close'] - data['close'].shift(1) ) / data['close'].shift(1)
data['rsi_14_roc'] = ( data['rsi_14'] - data['rsi_14'].shift(1) ) / data['rsi_14'].shift(1)
data['ma_9_roc'] = ( data['ma_9'] - data['ma_9'].shift(1) ) / data['ma_9'].shift(1)

data['close_roll'] = data['close'].rolling(window=14).mean()
data['rsi_14_roll'] = data['rsi_14'].rolling(window=14).mean()
data['ma_9_roll'] = data['ma_9'].rolling(window=14).mean()

data.dropna(axis=0, inplace=True)

In [3]:
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Define your feature space (list of feature names)
feature_space = ['rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'close_roc', 'rsi_14_roc', 'ma_9_roc', 'close_roll', 'rsi_14_roll', 'ma_9_roll']

In [4]:
inputs = data.iloc[:, [6,7,8,9,10,11,12,13,14,15,16,17]]
target = data.iloc[:, [5]]

In [5]:
data.shape

(16216, 18)

In [6]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=1, shuffle=False)

In [7]:
best_features = None
best_score = float('inf')

In [8]:
# Scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
nb = 0
for k in range(9, len(feature_space) + 1):
    for subset in combinations(feature_space, k):
        nb += 1
        print(subset)
print(nb)

('rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'close_roc', 'rsi_14_roc', 'ma_9_roc')
('rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'close_roc', 'rsi_14_roc', 'close_roll')
('rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'close_roc', 'rsi_14_roc', 'rsi_14_roll')
('rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'close_roc', 'rsi_14_roc', 'ma_9_roll')
('rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'close_roc', 'ma_9_roc', 'close_roll')
('rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'close_roc', 'ma_9_roc', 'rsi_14_roll')
('rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'close_roc', 'ma_9_roc', 'ma_9_roll')
('rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'close_roc', 'close_roll', 'rsi_14_roll')
('rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'close_roc', 'close_roll', 'ma_9_roll')
('rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'close_roc', 'rsi_14_roll', 'ma_9_roll')
('rsi_14', 'ma_9', 'macd', 'signal', 'hist', 'adx', 'rsi_14_roc', 'ma_9_roc', 

In [26]:
378 * 100

37800

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import xgboost

for k in range(1, len(feature_space) + 1):
    for subset in combinations(feature_space, k):
        # Define and train the LSTM model
        model = Sequential([
            Dense(64, activation='relu', input_shape=(k,)),  # Input layer with 10 features
            Dense(64, activation='relu'),                     # Hidden layer with 64 units
            Dense(1)                                          # Output layer with 1 unit for regression
        ])
        
        # Compile the model
        model.compile(optimizer='adam', loss='mse')  # Using Mean Squared Error loss for regression
        
        # Train the model
        model.fit(X_train[list(subset)], y_train, epochs=50, batch_size=32)

        # Evaluate model on validation set
        y_pred = model.predict(X_test[list(subset)])
        score = mean_squared_error(y_test, y_pred)

        # Update best feature combination if score improves
        if score < best_score:
            best_score = score
            best_features = subset

print("Best feature combination:", best_features)
print("Best MSE:", best_score)

Epoch 1/50
406/406 [==============================] - 2s 2ms/step - loss: 3878351.2500
Epoch 2/50
406/406 [==============================] - 1s 2ms/step - loss: 1180411.3750
Epoch 3/50
406/406 [==============================] - 1s 3ms/step - loss: 1176613.6250
Epoch 4/50
406/406 [==============================] - 1s 3ms/step - loss: 1171676.8750
Epoch 5/50
406/406 [==============================] - 1s 3ms/step - loss: 1166131.7500
Epoch 6/50
406/406 [==============================] - 1s 3ms/step - loss: 1160372.3750
Epoch 7/50
406/406 [==============================] - 2s 4ms/step - loss: 1153393.8750
Epoch 8/50
406/406 [==============================] - 1s 3ms/step - loss: 1146694.0000
Epoch 9/50
406/406 [==============================] - 1s 3ms/step - loss: 1139210.6250
Epoch 10/50
406/406 [==============================] - 1s 3ms/step - loss: 1131462.0000
Epoch 11/50
406/406 [==============================] - 2s 4ms/step - loss: 1122442.5000
Epoch 12/50
406/406 [====================

#### footnote:

In [ ]:
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Define your feature space (list of feature names)
feature_space = ['feature1', 'feature2', 'feature3', ...]

# Split your data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Best features flag
best_features = None
best_score = float('inf')

# Iterate over all possible feature combinations
for k in range(1, len(feature_space) + 1):
    for subset in combinations(feature_space, k):
        # Train model using only selected features
        model = LinearRegression()
        model.fit(X_train[list(subset)], y_train)
        
        # Evaluate model on validation set
        y_pred = model.predict(X_val[list(subset)])
        score = mean_squared_error(y_val, y_pred)
        
        # Update best feature combination if score improves
        if score < best_score:
            best_score = score
            best_features = subset

print("Best feature combination:", best_features)
print("Best MSE:", best_score)
